This notebook follow [this article](https://sgugger.github.io/convolution-in-depth.html) to try and recreate CNN from scratch to further understand the working of it.

In [1]:
import numpy as np

In [2]:
def arr2vec(x, kernel_size, stride=1, padding=0):
    k1, k2 = kernel_size
    mb, ch, n1, n2 = x.shape
    # initialize the empty output
    y = np.zeros((mb, ch, n1 + 2*padding, n2 + 2*padding))
    # since padding starts with 0, we can start the grid like this
    y[:, :, padding: n1 + padding, padding: n2 + padding] = x
    # initialize the starting indexes for the grid
    start_idx = np.array([j + (n2 + 2*padding) * i for i in range(0, n1-k1+1+2*padding, stride) for j in range(0,n2-k2+1+2*padding, stride)])
    #  we just want the numbers j+i∗n2 where j goes from 0 to k1, i goes from 0 to k2 and n2 is the number of columns
    # determine all the possible start indexes 
    # which correspond to the points with coordinates (i,j) 
    # where i can vary from 0 to n1−k1 and j can vary from 0 to n2−k2.
    # duplicate the grid ch times, adding n1×n2 each time we do
    grid = np.array([j + (n2+2*padding) * i + (n1+2*padding) * (n2+2*padding) * k for k in range(0, ch) for i in range(k1) for j in range(k2)])
    to_take = start_idx[:, None] + grid[None, :]
    batch = np.array(range(0, mb)) * ch * (n1+2*padding) * (n2+2*padding)
    return y.take(batch[:, None, None] + to_take[None, :, :])

In [3]:
def vec2arr(x, kernel_size, old_shape, stride=1, padding=0):
    k1, k2 = kernel_size
    n, p = old_shape
    mb, md, ftrs = x.shape
    ch = ftrs // (k1 * k2)
    # check how many windows could have passed over the element with coordinates (i,j)
    # that's all the windows that started at (i−k1i,j−k2j) 
    # where k1i can go from 0 to k1−1 and k2j can go from 0 to k2−1.
    idx = np.array([[[i-k1i, j-k2j] for k1i in range(k1) for k2j in range(k2)] for i in range(n) for j in range(p)])
    in_bounds = (idx[:, :, 0] >= -padding) * (idx[:, :, 0] <= n-k1+padding)
    in_bounds *= (idx[:, :, 1] >= -padding) * (idx[:, :, 0] <= p-k2+padding)

    # the second array is a boolean array that 
    # checks that the corners of our windows are inside the picture
    # taking the padding into account.
    in_strides = ((idx[:, :, 0] + padding) % stride == 0) * ((idx[:, :, 1] + padding) % stride == 0)

    # convert the couples into indexes and our channel dimension at the bottom
    to_take = np.concatenate([idx[:, :, 0] * k2 + idx[:, :, 1] + k1*k2*c for c in range(ch)], axis=0)

    #  we read on a line (when it's in bound and in-stride) the indexes of the elements to pick in each column
    to_take = to_take + np.array([ftrs * i for i in range(k1 * k2)])
    # For this to correspond to the indexes of the element in the array
    # we have to add to each column a multiple of the number of columns in our input
    to_take = np.concatenate([to_take + md*ftrs*m for m in range(mb)], axis=0)

    # add all the mini-batches over the same dimension    
    in_bounds = np.tile(in_bounds * in_strides, (ch * mb, 1))
    
    # use vectorization, we will create a big array of numbers, with as many row as as in our input, so N=m×ch×n1×n2 and as many columns as necessary.
    return np.where(in_bounds, np.take(x, to_take), 0).sum(axis=1).reshape(mb, ch, n, p)


In [4]:
class Convolution():
    def __init__(self, nc_in, nc_out, kernel_size, stride=2, padding=1):
        """
        nc_in: number of channel in
        nc_out: number of channel out
        kernel_size: size of the kernel
        stride: size of each step
        padding: outter padding for the input
        """
        self.kernel_size = kernel_size
        # weight = (num_chan_in * kernel size, num_chan_out) / sqrt(2 / num_chan_in)
        # it is initialized by following a normal distribution with a standard deviation of sqrt(2/num_chan_in)
        self.weights = np.random.randn(nc_in * kernel_size[0] * kernel_size[1], nc_out) * np.sqrt(2/nc_in)
        self.biases = np.zeros(nc_out)
        self.stride = stride
        self.padding = padding

    def forward(self, x):
        mb, ch, n, p = x.shape
        # mb: mini batch
        # ch: channel
        # n, p: w and height
        y = np.matmul(arr2vec(x, self.kernel_size, self.stride, self.padding), self.weights) + self.biases
        # inverting the last two axis
        y = np.transpose(y, (0, 2, 1))
        n1 = (n - self.kernel_size[0] + 2 * self.padding) // self.stride + 1
        p1 = (p - self.kernel_size[1] + 2 * self.padding) // self.stride + 1
        # reshaping the output size
        return y.reshape(mb, self.biases.shape[0], n1, p1)

    def backward(self, grad):
        mb, ch_out, n1, p1 = grad.shape
        # invert the last two axis
        grad = np.transpose(grad.reshape(mb, ch_out, n1*p1), (0, 2, 1))
        # calculate gradient wrt bias
        self.grad_b = grad.sum(axis=1).mean(axis=0)
        # calculate gradient wrt weight
        self.grad_w = (np.matmul(self.old_x[:, :, :, None], grad[:, :, None, :])).sum(axis=1).mean(axis=0)
        # calculate new grad
        new_grad = np.matmul(grad, self.weights.transpose())
        # reshape the gradient of the loss wrt to vec(x)
        return vec2arr(new_grad, self.kernel_size, self.old_size, self.stride, self.padding)